### Pull data from the live database

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sqlalchemy import create_engine

# Define your Azure SQL connection details
server = '1-10intex.database.windows.net'
database = 'Movies'
username = 'pigadmin'
password = 'Superduperstrongpassword!'
driver = 'ODBC Driver 18 for SQL Server'  # Or other valid installed driver

# Encode the driver string for URL
driver_encoded = driver.replace(' ', '+')


# Create the connection URL
connection_string = f'mssql+pyodbc://{username}:{password}@{server}:1433/{database}?driver={driver_encoded}&Encrypt=yes&TrustServerCertificate=no&Connection Timeout=30'

# Create SQLAlchemy engine
engine = create_engine(connection_string)

query = "SELECT * FROM movies_titles"
df = pd.read_sql(query, engine)
df.head()



,show_id,type,title,director,cast,country,release_year,rating,duration,description,...,musicals,nature_tv,reality_tv,spirituality,tv_action,tv_comedies,tv_dramas,talk_shows_tv_comedies,thrillers,genre
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,"Michael Hilow, Ana Hoffman, Dick Johnson, Kirs...",United States,2020,PG-13,90 min,As her father nears the end of his life filmma...,...,0,0,0,0,0,0,0,0,0,Documentaries
1,s2,TV Show,Blood & Water,None,Ama Qamata Khosi Ngema Gail Mabalane Thabang M...,South Africa,2021,TV-MA,2 Seasons,After crossing paths at a party a Cape Town te...,...,0,0,0,0,0,0,1,0,0,"Dramas, TV Dramas"
2,s3,TV Show,Ganglands,Julien Leclercq,Sami Bouajila Tracy Gotoas Samuel Jouy Nabiha ...,None,2021,TV-MA,1 Season,To protect his family from a powerful drug lor...,...,0,0,0,0,1,0,0,0,0,"Action, Adventure, TV Action"
3,s4,TV Show,Jailbirds New Orleans,None,None,None,2021,TV-MA,1 Season,Feuds flirtations and toilet talk go down amon...,...,0,0,1,0,0,0,0,0,0,"Docuseries, Reality TV"
4,s5,TV Show,Kota Factory,None,Mayur More Jitendra Kumar Ranjan Raj Alam Khan...,India,2021,TV-MA,2 Seasons,In a city of coaching centers known to train I...,...,0,0,0,0,0,1,0,0,0,"Comedies, TV Comedies"


### Look at the data and what it is missing

In [4]:
print(df.isna().sum())
print(df.shape)
print(df.columns)

show_id                                                  0
type                                                     0
title                                                    0
director                                              2114
cast                                                   119
country                                                814
release_year                                             0
rating                                                   4
duration                                                 1
description                                              0
action                                                   0
adventure                                                0
anime_series_international_tv_shows                      0
british_tv_shows_docuseries_international_tv_shows       0
children                                                 0
comedies                                                 0
comedies_dramas_international_movies                    

### Create the TFID vecoritzer and take out the stopwords

In [7]:
# Fill missing values with empty strings to prevent errors
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('')
df['description'] = df['description'].fillna('')
df['rating'] = df['rating'].fillna('')
df['release_year'] = str(df['release_year'].fillna(''))
df['genre'] = df.apply(lambda row: ' '.join([col for col in df.columns if df.columns.get_loc(col) >= 11 and row[col] == 1]), axis=1)

# Create a "soup" of combined text features
df['soup'] = df['cast'] + ' ' + df['director'] + ' ' + df['description'] + ' ' + df['genre'] + ' ' + df['rating'] + ' ' + df['release_year']

# Create a TfidfVectorizer and Remove stopwords
tfidf = TfidfVectorizer(stop_words='english')

# Fit and transform the data to a tfidf matrix
tfidf_matrix = tfidf.fit_transform(df['soup'])

# Print the shape of the tfidf_matrix
print(tfidf_matrix.shape)

# Preview the matrix by placing it into a DataFrame (which we won't need later)
df_tfidf = pd.DataFrame(tfidf_matrix.T.todense(), index=tfidf.get_feature_names_out(), columns=df['soup'])
df_tfidf

(8508, 50574)


soup,"Michael Hilow, Ana Hoffman, Dick Johnson, Kirsten Johnson, Chad Knorr Kirsten Johnson As her father nears the end of his life filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable. documentaries PG-13 0 2020\n1 2021\n2 2021\n3 2021\n4 2021\n ... \n8503 2007\n8504 2018\n8505 2009\n8506 2006\n8507 2015\nName: release_year, Length: 8508, dtype: int64","Ama Qamata Khosi Ngema Gail Mabalane Thabang Molaba Dillon Windvogel Natasha Thahane Arno Greeff Xolile Tshabalala Getmore Sithole Cindy Mahlangu Ryle De Morny Greteli Fincham Sello Maake Ka-Ncube Odwa Gwanya Mekaila Mathys Sandi Schultz Duane Williams Shamilla Miller Patrick Mofokeng After crossing paths at a party a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth. dramas tv_dramas TV-MA 0 2020\n1 2021\n2 2021\n3 2021\n4 2021\n ... \n8503 2007\n8504 2018\n8505 2009\n8506 2006\n8507 2015\nName: release_year, Length: 8508, dtype: int64","Sami Bouajila Tracy Gotoas Samuel Jouy Nabiha Akkari Sofia Lesaffre Salim Kechiouche Noureddine Farihi Geert Van Rampelberg Bakary Diombera Julien Leclercq To protect his family from a powerful drug lord skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war. adventure tv_action TV-MA 0 2020\n1 2021\n2 2021\n3 2021\n4 2021\n ... \n8503 2007\n8504 2018\n8505 2009\n8506 2006\n8507 2015\nName: release_year, Length: 8508, dtype: int64","Feuds flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series. docuseries reality_tv TV-MA 0 2020\n1 2021\n2 2021\n3 2021\n4 2021\n ... \n8503 2007\n8504 2018\n8505 2009\n8506 2006\n8507 2015\nName: release_year, Length: 8508, dtype: int64","Mayur More Jitendra Kumar Ranjan Raj Alam Khan Ahsaas Channa Revathi Pillai Urvi Singh Arun Kumar In a city of coaching centers known to train India’s finest collegiate minds an earnest but unexceptional student and his friends navigate campus life. comedies tv_comedies TV-MA 0 2020\n1 2021\n2 2021\n3 2021\n4 2021\n ... \n8503 2007\n8504 2018\n8505 2009\n8506 2006\n8507 2015\nName: release_year, Length: 8508, dtype: int64","Kate Siegel Zach Gilford Hamish Linklater Henry Thomas Kristin Lehman Samantha Sloyan Igby Rigney Rahul Kohli Annarah Cymone Annabeth Gish Alex Essoe Rahul Abburi Matt Biedel Michael Trucco Crystal Balint Louis Oliver Mike Flanagan The arrival of a charismatic young priest brings glorious miracles ominous mysteries and renewed religious fervor to a dying town desperate to believe. dramas tv_dramas TV-MA 0 2020\n1 2021\n2 2021\n3 2021\n4 2021\n ... \n8503 2007\n8504 2018\n8505 2009\n8506 2006\n8507 2015\nName: release_year, Length: 8508, dtype: int64","Vanessa Hudgens Kimiko Glenn James Marsden Sofia Carson Liza Koshy Ken Jeong Elizabeth Perkins Jane Krakowski Michael McKean Phil LaMarr Robert Cullen José Luis Ucha Equestria's divided. But a bright-eyed hero believes Earth Ponies Pegasi and Unicorns should be pals — and hoof to heart she’s determined to prove it. children family_movies PG 0 2020\n1 2021\n2 2021\n3 2021\n4 2021\n ... \n8503 2007\n8504 2018\n8505 2009\n8506 2006\n8507 2015\nName: release_year, Length: 8508, dtype: int64","Kofi Ghanaba Oyafunmike Ogunlano Alexandra Duah Nick Medley Mutabaruka Afemo Omilami Reggie Carter Mzuri Haile Gerima On a photo shoot in Ghana an American model slips back in time becomes enslaved on a plantation and bears witness to the agony of her ancestral past. dramas TV-MA 0 2020\n1 2021\n2 2021\n3 2021\n4 2021\n ... \n8503 2007\n8504 2018\n8505 2009\n8506 2006\n8507 2015\nName: release_year, Length: 8508, dtype: int64","Mel Giedroyc Sue Perkins Mary Berry Paul Hollywood Andy Devonshire A talented batch of amateur bakers face off in a 10-week competition whipping up their best dishes in the hopes of being named the U.K.'s best. reality_tv TV-14 0 2020\n1 2021\n2 2021\n3 2021\n4

### Find the cosine similarity scores and put them in a df

In [9]:
# Compute the cosine similarity between each movie description
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# For easier viewing, put it in a dataframe
df_results = pd.DataFrame(cosine_sim, columns=df['show_id'], index = df['show_id'])
# df_results.to_csv("content_filtering_recommendations.csv")
# df_results.sort_values(by=['s8'], ascending=False)
df_results

show_id,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,...,s8798,s8799,s8800,s8801,s8802,s8803,s8804,s8805,s8806,s8807
show_id,,,,,,,,,,,,,,,,,,,,,
s1,1.000000,0.012170,0.015679,0.024921,0.024012,0.018817,0.025991,0.016239,0.039346,0.034523,...,0.026624,0.016602,0.017305,0.019943,0.017401,0.017492,0.031845,0.019215,0.034842,0.017499
s2,0.012170,1.000000,0.013395,0.021291,0.015169,0.023504,0.025484,0.015385,0.015829,0.013932,...,0.024609,0.023985,0.042352,0.018345,0.025138,0.014347,0.040061,0.013972,0.013025,0.014939
s3,0.015679,0.013395,1.000000,0.027431,0.019543,0.015096,0.037322,0.017874,0.020394,0.015913,...,0.019018,0.018274,0.016960,0.016013,0.019153,0.016387,0.031210,0.018001,0.016782,0.023896
s4,0.024921,0.021291,0.027431,1.000000,0.031063,0.023994,0.024300,0.028410,0.052261,0.025293,...,0.030227,0.029045,0.026957,0.025451,0.030442,0.026046,0.049606,0.028612,0.037111,0.027258
s5,0.024012,0.015169,0.019543,0.031063,1.000000,0.017095,0.017313,0.020241,0.023094,0.027519,...,0.021536,0.036809,0.019206,0.029826,0.021689,0.018557,0.055779,0.040291,0.053234,0.033163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
s8803,0.017492,0.014347,0.016387,0.026046,0.018557,0.016168,0.027459,0.019144,0.021747,0.020025,...,0.020280,0.037221,0.038796,0.019261,0.027924,1.000000,0.033282,0.044893,0.040614,0.020629
s8804,0.031845,0.040061,0.031210,0.049606,0.055779,0.054016,0.042799,0.032324,0.041419,0.037966,...,0.077117,0.060171,0.034446,0.032522,0.057280,0.033282,1.000000,0.042949,0.052933,0.034831
s8805,0.019215,0.013972,0.018001,0.028612,0.040291,0.015746,0.018737,0.018644,0.023890,0.046656,...,0.022278,0.022391,0.019868,0.034834,0.019977,0.044893,0.042949,1.000000,0.063781,0.020090


### Build the csv file that holds all the recommendations

In [ ]:
# Create a DataFrame to hold the top 20 recommendations for each show
top_n = 20
top_recommendations = {}

for show_id in df['show_id']:
    similar_scores = df_results[show_id].drop(show_id)  # Drop self-match
    top_similar = similar_scores.sort_values(ascending=False).head(top_n).index.tolist()
    top_recommendations[show_id] = top_similar

# Convert to a DataFrame
recommendations_df = pd.DataFrame.from_dict(top_recommendations, orient='index')
recommendations_df.index.name = 'show_id'
recommendations_df = recommendations_df.reset_index()
display(recommendations_df)

### Put it in the Azure database

In [12]:
# Define your Azure SQL connection details
server = '1-10intex.database.windows.net'
database = 'Movies'
username = 'pigadmin'
password = 'Superduperstrongpassword!'
driver = 'ODBC Driver 18 for SQL Server'  # Or another valid installed driver

# Encode the driver string for URL
driver_encoded = driver.replace(' ', '+')

# Create the connection URL
connection_string = f'mssql+pyodbc://{username}:{password}@{server}:1433/{database}?driver={driver_encoded}&Encrypt=yes&TrustServerCertificate=no&Connection Timeout=30'

# Create SQLAlchemy engine
engine = create_engine(connection_string)

# Upload DataFrame to Azure
recommendations_df.to_sql(
    'content_filtering_recommendations',
    con=engine,
    if_exists='replace',  # or 'append' if you want to add new data without replacing
    index=False,
    chunksize=1000
)

93